# OrientDB tutorial

## Prerequisites

To run Orientdb :
* Open a terminal
* In the terminal :
```bash
cd /opt/orientdb
./bin/server.sh
```
* On the terminal, you will be prompted to define your `root` password. Choose one as long as you remember it :)

### Documentation

You will find all documentation for :
* [OrientDB SQL reference](http://www.orientdb.com/docs/last/SQL-Functions.html)
* [Orientdb python client](http://orientdb.com/docs/last/PyOrient-Client.html#working-with-the-client)

## Import libraries

In [1]:
import pyorient

In [2]:
ROOT_PASSWORD = "root"
client = pyorient.OrientDB("localhost", 2424)
session_id = client.connect("root", ROOT_PASSWORD)

In [3]:
print(client.db_list())

{{'databases': {'GratefulDeadConcerts': 'plocal:/opt/orientdb/databases/GratefulDeadConcerts'}}}


## I. Quick start

### Creating the database

**Q:** Create a database `gods` as a `GRAPH_DATABASE` in `MEMORY_STORAGE_TYPE`. 

We will use it to store relationships between Greek deities.

In [4]:
client.db_create("gods", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)

**Q:** Connect your pyorient client to the `gods` database.

In [5]:
client.db_open( "gods", "root", ROOT_PASSWORD) 

**Q:** You should now be able to launch OrientDB queries through the Python client with the [command()](http://orientdb.com/docs/last/PyOrient-Client-Command.html) function. 

You should think of OrientDB as a Graph-Document database for the following questions. Each vertex and edge will contain information on it inside a JSON document.

Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`. The [CREATE VERTEX : Create a vertex using JSON content](http://orientdb.com/docs/last/SQL-Create-Vertex.html) doc page should help you.

In [6]:
client.command("create vertex content {name: 'Zeus', symbol: 'thunder'}") 

You have created a VERTEX in the previous question. The VERTEX is a [class](https://orientdb.com/docs/last/Tutorial-Classes.html) of OrientDB which defines a record that can be linked to others through EDGE instances.

You can find all VERTEX created in the database with a SQL command on the `V` table, like `SELECT * FROM V`. 

**Q:** Print all current vertices in `gods`, it should only have `Zeus` though for now.

In [7]:
for val in client.command("SELECT * FROM V"):
    print(val)

{'@V':{'name': 'Zeus', 'symbol': 'thunder'},'version':1,'rid':'#9:0'}


**Q:** Create new vertices with content : 
```
{name:Héra, symbol:tiara}
{name:Poséidon, symbol:trident}
{name:Athena, symbol:helmet}
{name:Arès, symbol:weapons} 
```

In [8]:
client.command("create vertex content {name:'Héra', symbol:'tiara'}")
client.command("create vertex content {name:'Poséidon', symbol:'trident'}") 
client.command("create vertex content {name:'Athena', symbol:'helmet'}")
client.command("create vertex content {name:'Arès', symbol:'weapons'} ") 

**Q:** Display all vertices with name = `Arès`

In [9]:
for val in client.command("SELECT * FROM V WHERE name = 'Arès'"):
    print(val)

{'@V':{'name': 'Arès', 'symbol': 'weapons'},'version':1,'rid':'#9:2'}


**Q:** Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}

In [10]:
client.command("create edge from #9:0 to #9:1 content {kind: 'sibling'}")

**Q:** Redisplay all vertices, discuss.

In [11]:
for val in client.command("SELECT * FROM V"):
    print(val)

{'@V':{'name': 'Zeus', 'symbol': 'thunder', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x7faeca54b908>},'version':2,'rid':'#9:0'}
{'@V':{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7faeca54ba90>},'version':2,'rid':'#9:1'}
{'@V':{'name': 'Arès', 'symbol': 'weapons'},'version':1,'rid':'#9:2'}
{'@V':{'name': 'Héra', 'symbol': 'tiara'},'version':1,'rid':'#10:0'}
{'@V':{'name': 'Athena', 'symbol': 'helmet'},'version':1,'rid':'#10:1'}


**Q:** Display all edges. They are contained in the class `E`

In [12]:
for val in client.command("SELECT * FROM E"):
    print(val)

{'@E':{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x7faeca54bba8>, 'in': <pyorient.otypes.OrientRecordLink object at 0x7faeca54bb00>},'version':1,'rid':'#11:0'}


Two fields on vertices have appeared, containing the outgoing (out_) and incoming (in_) links.

At the edge level, two fields point to the original (out) and destination (in) vertices.

**Q:** Lets create some more edges :

* Zeus > Héra (sibling)
* Zeus > Arès (father)
* Zeus > Athena (father)
* Héra > Arès (mother)
* Héra > Zeus (sibling)
* Poséidon > Zeus (sibling)

_Hint 1 :_ check [the CREATE EDGE doc page](http://orientdb.com/docs/last/SQL-Create-Edge.html) to find an example for creating edges on vertices using subqueries so you can run queries to fetch the vertices before creating an edge in between.

_Hint 2 :_ after you have found the command to create edges between vertices with sub-queries, you should be well-versed enough in Python to create a list of all edges in the question, and loop the command on each element of the list to create all edges in one go =)

In [13]:
for i,o,c in [
    ('Zeus','Héra','sibling'),
    ('Zeus','Arès','father'),
    ('Zeus','Athena','father'),
    ('Héra','Arès','mother'),
    ('Héra','Zeus','sibling'),
    ('Poséidon','Zeus','sibling')
]:
    command=f"create edge from (select from V where name = '{i}') to (select from V where name = '{o}') content {{kind: '{c}'}}"
    print(command)
    client.command(command)

create edge from (select from V where name = 'Zeus') to (select from V where name = 'Héra') content {kind: 'sibling'}
create edge from (select from V where name = 'Zeus') to (select from V where name = 'Arès') content {kind: 'father'}
create edge from (select from V where name = 'Zeus') to (select from V where name = 'Athena') content {kind: 'father'}
create edge from (select from V where name = 'Héra') to (select from V where name = 'Arès') content {kind: 'mother'}
create edge from (select from V where name = 'Héra') to (select from V where name = 'Zeus') content {kind: 'sibling'}
create edge from (select from V where name = 'Poséidon') to (select from V where name = 'Zeus') content {kind: 'sibling'}


### Looking for data

**Q:** Using [out()](http://orientdb.com/docs/last/Tutorial-Working-with-graphs.html#querying-graphs) function, display all vertices connected and outgoing from Zeus.

You should use the EXPAND() special function to transform the vertex collection in the result-set by expanding it, making the results more readable.

In [14]:
for val in client.command("select out() from V where name = 'Zeus'"):
    print(val)

{{'out': [<pyorient.otypes.OrientRecordLink object at 0x7faeca555550>, <pyorient.otypes.OrientRecordLink object at 0x7faeca5555c0>, <pyorient.otypes.OrientRecordLink object at 0x7faeca555668>, <pyorient.otypes.OrientRecordLink object at 0x7faeca5556d8>]},'version':0,'rid':'#-2:0'}


In [15]:
for val in client.command("select expand(out()) from V where name = 'Zeus'"):
    print(val)

{'@V':{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7faeca555908>, 'out_': <pyorient.otypes.OrientBinaryObject object at 0x7faeca5558d0>},'version':3,'rid':'#9:1'}
{'@V':{'out_': <pyorient.otypes.OrientBinaryObject object at 0x7faeca5550f0>, 'name': 'Héra', 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7faeca555208>},'version':4,'rid':'#10:0'}
{'@V':{'in_': <pyorient.otypes.OrientBinaryObject object at 0x7faeca5552e8>, 'name': 'Arès', 'symbol': 'weapons'},'version':3,'rid':'#9:2'}
{'@V':{'name': 'Athena', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7faeca555cc0>},'version':2,'rid':'#10:1'}


**Q:** Display all vertices which got a father (the vertices which are the destination of an arc whose kind attribute is father).

_Hint: You can notice that we use the field `in` the arc, and not the function `in()` which applies to vertices._

In [16]:
for val in client.command("select expand(in) from E where kind = 'father'"):
    print(val)

{'@V':{'in_': <pyorient.otypes.OrientBinaryObject object at 0x7faeca555160>, 'name': 'Arès', 'symbol': 'weapons'},'version':3,'rid':'#9:2'}
{'@V':{'name': 'Athena', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7faeca555550>},'version':2,'rid':'#10:1'}


**Q:** As in SQL, the operator `in` used in a clause `where` allows to restrict the possible values with an embedded query _(where ... in (select ...))_. 

Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.

In [17]:
for val in client.command("select from V where out_ in (select from E where kind = 'mother')"):
    print(val)

{'@V':{'out_': <pyorient.otypes.OrientBinaryObject object at 0x7faeca5555f8>, 'name': 'Héra', 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7faeca5556a0>},'version':4,'rid':'#10:0'}


**Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).

In [18]:
for val in client.command("select expand(in) from E where kind = 'sibling' and out in (select from V where name = 'Zeus')"):
    print(val)

{'@V':{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7faeca54b3c8>, 'out_': <pyorient.otypes.OrientBinaryObject object at 0x7faeca54ba20>},'version':3,'rid':'#9:1'}
{'@V':{'out_': <pyorient.otypes.OrientBinaryObject object at 0x7faeca54be80>, 'name': 'Héra', 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7faeca54b4a8>},'version':4,'rid':'#10:0'}


## Modeling a Product Recommendation System

You are currently modeling the data of a product recommendation system with OrientDB.

The main purpose of such a system is to answer the question "which products were purchased by their people who purchased product X? »

Purchased products have only one name field. They are purchased by people who have a nickname.

When a person buys a product, the date of purchase is stored. 

Instead of working with "anonymous" vertices and arcs, you will use classes. The `create class` command allows you to create custom classes.

The vertex classes must extend V, the arc classes must extend E.

**Q:** Create an `eCommerce` database, and the necessary classes to model the system.

PS : you can view all classes in the database with :

```python
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)
```

In [19]:
client.db_create("eCommerce", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)
client.db_open( "eCommerce", "root", ROOT_PASSWORD) 

client.command("create class Product extends V")
client.command("create class Person extends V")
client.command("create class Bought extends E")

for v in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(v)

{{'name': 'OTriggered'},'version':0,'rid':'#-2:0'}
{{'name': 'OIdentity'},'version':0,'rid':'#-2:1'}
{{'name': 'Bought'},'version':0,'rid':'#-2:2'}
{{'name': 'OUser'},'version':0,'rid':'#-2:3'}
{{'name': 'V'},'version':0,'rid':'#-2:4'}
{{'name': 'E'},'version':0,'rid':'#-2:5'}
{{'name': 'ORestricted'},'version':0,'rid':'#-2:6'}
{{'name': 'OFunction'},'version':0,'rid':'#-2:7'}
{{'name': 'OSchedule'},'version':0,'rid':'#-2:8'}
{{'name': 'Product'},'version':0,'rid':'#-2:9'}
{{'name': 'OSequence'},'version':0,'rid':'#-2:10'}
{{'name': 'Person'},'version':0,'rid':'#-2:11'}
{{'name': 'ORole'},'version':0,'rid':'#-2:12'}


**Q:** Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.

In [20]:
for p in ["spaghetti", "bolognese sauce", "cheese", "apple"]:
    client.command(f"create vertex product content {{name: '{p}'}}")

**Q:** Create the following people: `peter`, `meredith`.

In [21]:
for p in ["peter", "meredith"]:
    client.command(f"create vertex person content {{pseudo: '{p}'}}")

**Q:** Create the following purchases: 
- peter > spaghetti + cheese on 20/01/2016 
- meredith > cheese + apple + bolognese sauce on 22/01/2016
- peter > spaghetti + bolognese sauce on 27/01/2016


In [22]:
for person,product,date in [
    ('peter','spaghetti','20/01/2016'),
    ('peter','cheese','20/01/2016'),
    ('meredith','cheese','22/01/2016'),
    ('meredith','apple','22/01/2016'),
    ('meredith','bolognese sauce','22/01/2016'),
    ('peter','spaghetti','27/01/2016'),
    ('peter','bolognese sauce','27/01/2016')
]:
    command=f"create edge bought from (select from person where pseudo = '{person}') to (select from product where name = '{product}') content {{date: '{date}'}}"
    print(command)
    client.command(command)

create edge bought from (select from person where pseudo = 'peter') to (select from product where name = 'spaghetti') content {date: '20/01/2016'}
create edge bought from (select from person where pseudo = 'peter') to (select from product where name = 'cheese') content {date: '20/01/2016'}
create edge bought from (select from person where pseudo = 'meredith') to (select from product where name = 'cheese') content {date: '22/01/2016'}
create edge bought from (select from person where pseudo = 'meredith') to (select from product where name = 'apple') content {date: '22/01/2016'}
create edge bought from (select from person where pseudo = 'meredith') to (select from product where name = 'bolognese sauce') content {date: '22/01/2016'}
create edge bought from (select from person where pseudo = 'peter') to (select from product where name = 'spaghetti') content {date: '27/01/2016'}
create edge bought from (select from person where pseudo = 'peter') to (select from product where name = 'bologne

**Q:** Who bought Bolognese sauce?

In [23]:
for val in client.command("select expand(in()) from Product where name = 'bolognese sauce'"):
    print(val)

{'@Person':{'out_Bought': <pyorient.otypes.OrientBinaryObject object at 0x7faeca5550f0>, 'pseudo': 'meredith'},'version':4,'rid':'#16:0'}
{'@Person':{'out_Bought': <pyorient.otypes.OrientBinaryObject object at 0x7faeca555da0>, 'pseudo': 'peter'},'version':5,'rid':'#15:0'}


**Q:** It is possible to link the `out` and `in` navigation functions. What products are purchased with Bolognese sauce? 

In [24]:
for val in client.command("select expand(in().out()) from Product where name = 'bolognese sauce'"):
    print(val)

{'@Product':{'in_Bought': <pyorient.otypes.OrientBinaryObject object at 0x7faeca555b38>, 'name': 'cheese'},'version':3,'rid':'#13:1'}
{'@Product':{'name': 'apple', 'in_Bought': <pyorient.otypes.OrientBinaryObject object at 0x7faeca54b550>},'version':2,'rid':'#14:1'}
{'@Product':{'in_Bought': <pyorient.otypes.OrientBinaryObject object at 0x7faeca54be80>, 'name': 'bolognese sauce'},'version':3,'rid':'#14:0'}
{'@Product':{'in_Bought': <pyorient.otypes.OrientBinaryObject object at 0x7faeca54bda0>, 'name': 'spaghetti'},'version':3,'rid':'#13:0'}
{'@Product':{'in_Bought': <pyorient.otypes.OrientBinaryObject object at 0x7faeca54bdd8>, 'name': 'cheese'},'version':3,'rid':'#13:1'}
{'@Product':{'in_Bought': <pyorient.otypes.OrientBinaryObject object at 0x7faeca5430f0>, 'name': 'spaghetti'},'version':3,'rid':'#13:0'}
{'@Product':{'in_Bought': <pyorient.otypes.OrientBinaryObject object at 0x7faeca543278>, 'name': 'bolognese sauce'},'version':3,'rid':'#14:0'}


## Postquisites

Since we create databases in memory, they get destroyed on server shutdown.